In [ ]:
import chess.gaviota
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run 11_imports.ipynb
%run 12_integer_management.ipynb
%run 13_mirroring.ipynb
%run 14_functions.ipynb

# Überprüfen der $S_n$ Mengen

Bei der Bestimmung der $S_n$ Mengen wurden alle möglichen Züge rückwärts durchgeführt.
Damit die Retrograde Analysis erfolgreich zum Gewinnen von Endspielsituationen angewandt werden kann, muss diese Berechnung erfolgreich gewesen sein.
Um sicherzustellen, dass die Situationen, die z. B. in $S_{10}$ zu finden sind, auch wirklich in $<=10$ Zügen beendet werden, wurden in diesem Notebook Funktionen definiert, die diesen Aspekt überprüfen. Hierfür werden zuerst die berechneten $S_n$ Mengen aus der Datei geladen und in nachkommenden Variablen gespeichert:
* `g_piece_list`: Die Piece List, die zur Berechnung der Mengen verwendet worden ist.
* `g_s_n_sequence` Die $S_n$-Mengen mit sowohl einzigartigen, als auch gespiegelten Stellungen.

In [ ]:
%%time
g_piece_list, g_s_n_sequence = load_data("queen_03_06")
print("Read File")
g_s_n_sequence = gen_all_integers(g_s_n_sequence, g_piece_list)

Die erste Funktion zur Überprüfung, ist ``int_in_lower_sequence(board_int, sequence, sequence_index)``. Diese überprüft für ein mitgegebenes Spielbrett in Integer-Repräsentation (``board_int``), ob diese in einer $S_n$ Menge liegt, für die gilt:
$$
\mathtt{n} < \mathtt{sequence\_index}
$$

Hierbei stellen die möglichen $S_n$-Mengen die Kombination aus den Parametern `sequence` und `sequence_index` dar ($S_n$ =  `sequence[sequence_index]`).

Die Funktion gibt hierfür einen booleschen Wert zurück.

In [ ]:
def int_in_lower_sequence(board_int, sequence, sequence_index):
    for s in sequence[:sequence_index]:
        if board_int in s:
            return True
    return False

Eine weitere Überprüfung liegt darin, dass keine der Schachbretter in eine zu niedrige $S_n$ Menge eingeordnet wurde. Dies wird mit der Funktion ``every_move_of_sequence_in_lower(sequence_index, s_n_sequence)`` umgesetzt. Hierfür erhält sie einen ``sequence_index``, die für die zu überprüfende $S_n$ Menge den Index bereitstellt. Die $S_n$-Menge kann daraufhin aus dem Parameter `s_n_sequence` entnommen werden. Für jede Spielsituation wird Folgendes überprüft:

$$
\forall \mathtt{board} \in \mathtt{S_n} : \mathtt{move} \in \mathtt{board.legal\_moves} \implies \mathtt{board.push(move)} \in \mathtt{S_m} \land m<n
$$

Als Ergebnis gibt die Funktion eine boolesche Variable zurück, die ``True`` zurückgibt, falls die Bedingung erfüllt ist, ansonsten ``False``.

In [ ]:
def every_move_of_sequence_in_lower(sequence_index):
    print("Checking S_" + str(sequence_index) + "...")
    n_integers = g_s_n_sequence[sequence_index]
    for board_int in n_integers:
        cur_board = to_board(board_int, g_piece_list)
        if not cur_board.turn:
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_int = to_integer(cur_board, g_piece_list)
                cur_board.pop()
                if not int_in_lower_sequence(cur_int, g_s_n_sequence, sequence_index):
                    print("Int: " + str(cur_int) + " not in S lower than " + str(sequence_index))
                    return False
                
        else:
            in_lower = False
            for move in cur_board.legal_moves:
                cur_board.push(move)
                try:
                    cur_int = to_integer(cur_board, g_piece_list)
                    if int_in_lower_sequence(cur_int, g_s_n_sequence, sequence_index):
                        in_lower = True
                        cur_board.pop()
                        break
                except ValueError:
                    # Catch wrong pawn conversions
                    pass
                cur_board.pop()
            if not in_lower:
                print(cur_board)
                print(to_integer(cur_board, g_piece_list))
                print("Cannot reach lower S_n")
                return False
    return True

Die folgende Zelle überprüft für jede $S_n$-Menge, ob sie den zuvor beschriebenen Test besteht.

In [ ]:
for i in range(len(g_s_n_sequence)):
    every_move_of_sequence_in_lower(i)

## Syzygy

Zusätzlich zum rechnerischen Überprüfen der Ergebnisse kann eine andere Methode zum Verifizieren verwendet werden. 
Schach-Endspieldatenbanken sind keine neue Erfindung und wurden bereits von anderen Forschern entwickelt.
Ein lang anhaltendes Projekt, welches gegenwärtig Datenbanken für Situationen mit bis zu 7 Figuren auf dem Spielfeld anbietet, ist [Syzygy](https://syzygy-tables.info/).
Das Projekt hält zwei Datentypen für alle Spielsituationen vor:
WDL Daten und DTZ Daten.

WDL steht für Win / Draw / Loss und gibt dem Nutzer eine Information über den Wert einer Spielsituation. 
Eine Anfrage an die Datenbank mit einer Situation wird mit einem der folgenden Werte beantwortet: -2, -1, 0, 1, 2.
Positive Werte implizieren, dass bei perfektem Spiel der aktuelle Spieler gewinnt, negative Werte bedeuten, dass der aktuelle Spieler verliert.
Die 0 bedeutet, dass das Spiel (wenn beide Seiten perfekt spielen) in einem Unentschieden endet.
Eine Zwei ist ein sicherer Sieg / Verlust, während eine Eins in einem Gewinn oder unentschieden mittels der 50-Zug Regel enden kann.

Die Interessantere der Dateien ist die DTZ-Datei.
DTZ steht für Distance to Zero. Die DTZ Tabelle enthält Werte von -100 bis 100. 
Positiv, Negativ und Null kann genau wie WDL interpretiert werden.
Die Zahlen von -100 bis -1 und 1 bis 100 geben die Anzahl der Halbzüge bis zu einem Gewinn (oder Reset der 50-Züge Regel) an.
Stetiges verringern einer positiven DTZ führt also zu einem Gewinn.

Die DTZ-Zahl einer Spielsituation kann mit dem $n$ verglichen werden, in welcher Menge $S_n$ diese Situation in der `.chessAI` eingeordnet wurde.
Stimmen diese Zahlen überein, war die Berechnung korrekt.

*Hinweis: Je nach Ausführung der Syzygy-Tabellen werden halbe oder ganze Züge gespeichert, es muss daher beim Vergleich eine Toleranz von einem $n$ akzeptiert werden.*    

Die Funktion `compare_with_syzygy(syzygy)` führt den Vergleich zwischen der Endspieldaten und Syzygy durch und gibt die Anzahl falsch eingeordneter Situationen zurück.
Sie erhält dafür ein Objekt vom Type `chess.syzygy` und die Liste von $S_n$-Mengen `s_n_sequence`.
Als Ergebnis, gibt sie die Anzahl der falschen Situationen `total_count` zurück. 

In [ ]:
def compare_with_syzygy(syzygy):
    total_count = 0
    
    for n in range(len(g_s_n_sequence)):
        n_count = 0
        for int in g_s_n_sequence[n]:
            chess_board = to_board(int, g_piece_list)
            if n != abs(syzygy.probe_dtz(chess_board)) != n + 1:
                n_count += 1
        print(f"S{n}: Syzygy believes {n_count} of {len(g_s_n_sequence[n])} Situations are wrongly placed in the sequence.")
        total_count += n_count
    print(f"Syzygy believes {total_count} Situations are wrongly placed in the sequence.")
    return total_count

Die Syzygy-Dateien müssen sich entweder im Ordner `./syzygy` befinden oder der Pfad angepasst werden.
Die folgende Zelle initialisiert das Objekt.

In [ ]:
syzygy = chess.syzygy.Tablebase()
syzygy.add_directory("./syzygy")

Durch einen Funktionsaufruf von `compare_with_syzygy(syzygy)` kann der Vergleich gestartet werden.

In [ ]:
compare_with_syzygy(syzygy)

## Gaviota

Als Alternative für Syzygy gibt es die Gaviota Endspieltabellen.
Diese können nach ihrer DTM Information ausgewertet werden. DTM steht hierbei für Depth to Mate und gibt an, wie viele Halbzüge eine Situation von einem Schach-Matt entfernt ist.
Ist die Situation unentschieden, ist die DTM Null. 
Für den Vergleich der KI mit Gaviota wird die Funktion `compare_with_gaviota(gaviota)` definiert.
Diese erhält als Parameter ein Objekt vom Typ `chess.Gaviota` und führt den Vergleich mit der globalen Liste von $S_n$ Mengen `g_s_n_sequence` durch.
Sie gibt in der Konsole eine Auswertung aus, und liefert die Anzahl falsch zugeordneter Boards als Rückgabewert zurück.

In [ ]:
def compare_with_gaviota(gaviota):
    total_count = 0
    
    for n in range(len(g_s_n_sequence)):
        n_count = 0
        for int_rep in g_s_n_sequence[n]:
            chess_board = to_board(int_rep, g_piece_list)
            if n != abs(gaviota.probe_dtm(chess_board)):
                n_count += 1
        print(f"S{n}: Gaviota believes {n_count} of {len(g_s_n_sequence[n])} Situations are wrongly placed in the sequence.")
        total_count += n_count
    print(f"Gaviota believes {total_count} Situations are wrongly placed in the sequence.")
    return total_count


Die Gaviota-Dateien müssen sich entweder im Ordner `./gaviota` befinden oder der Pfad angepasst werden.
Die folgende Zelle initialisiert das Objekt.

In [ ]:
gaviota = chess.gaviota.open_tablebase("./gaviota")

Durch einen Funktionsaufruf von ``compare_with_gaviota(gaviota)`` kann der Vergleich gestartet werden

In [ ]:
compare_with_gaviota(gaviota)

## Matching Rate
Eine weitere Betrachtung, welche über die Qualität der berechneten $S_n$ Mengen Auskunft gibt, ist der Anteil der Situationen, die mit den $S_n$-Mengen abgedeckt werden.
Für diesen Vergleich werden alle validen Stellungen mit den angegebenen Figuren betrachtet, und überprüft wie viele davon sich in einer $S_n$ Menge befinden.
Da nicht alle Stellungen zu einem Sieg geführt werden können, werden die zuvor erklärten Gaviota-Tabellen verwendet, um unentschiedene Situationen herauszufiltern.
Diese werden für den Anteil nicht beachtet.

Zunächst werden globale Variablen für den Vergleich definiert:
* `g_total`: Ein Integer, der die Anzahl aller möglichen validen Spielsituationen darstellt.
* `g_not_matched`: Ein Integer, der die Anzahl an nicht zugeordneten Spielsituationen darstellt.
* `g_all_boards`: Eine Menge von Integer-Repräsentationen, die im Rahmen der $S_n$-Berechnung bestimmt worden sind.

In [ ]:
g_total = 0
g_not_matched = 0
g_all_boards = set()

Anschließend wird die Funktion `fill_boards_with_piece(int_boards_set, piece, cur_piece_list, total, not_matched)`, welche bereits im Notebook `02_calculation_backend.ipynb` erklärt wurde, angepasst.
Diese Version fügt für Stellungen, welche alle Figuren beinhalten, zusätzlich einen Vergleich durch, ob sich die Integer-Repräsentation in der globalen Variable `g_all_boards` befindet.
Diese enthält alle Stellungen.
Ist dies nicht der Fall, wird überprüft, ob Gaviota einen DTM-Wert, welcher nicht 0 ist liefert.
Ist dies der Fall, zählt das Board als nicht eingeordnet und deutet auf einen Fehler der Berechnung hin.

In [ ]:
def fill_boards_with_piece(int_boards_set, piece, cur_piece_list, total, not_matched):
    finished_boards = set()
    all_squares = set(range(64))
    piece_count = len(g_piece_list)
    for int_board in int_boards_set:
        board_o = to_board(int_board, cur_piece_list)
        used_squares = set(board_o.piece_map().keys())
        for square in all_squares:
            if square not in used_squares:
                tmp_piece_list = cur_piece_list.copy()
                board_object = board_o.copy()
                board_object.set_piece_at(square, piece)
                tmp_piece_list.append(piece)
                board_int = to_integer(board_object, tmp_piece_list)
                if len(used_squares) > 1 and not board_object.is_valid():
                    continue

                if len(used_squares) + 1 < piece_count: #Board is valid, but needs more pieces
                    finished_boards.add(board_int)
                    continue
                
                total += 1
                if board_int not in g_all_boards:
                    if gaviota.probe_dtm(board_object) != 0: #0 is a draw and not thus relevant for this check
                        not_matched += 1
                    
    return finished_boards, total, not_matched


Die Funktion `matchrate_for_player(turn)` bestimmt die Abweichungen für den im Parameter `turn` übergebenen Spieler.
Da eine Stellung sich unterscheidet, abhängig welcher Spieler am Zug ist, müssen diese Rechnungen für beide Spieler durchgeführt werden.
Die Funktion baut sich wie die bereits in `02_calculation_backend.ipynb` erklärte Funktion `calculate_s0` auf.

In [ ]:
def matchrate_for_player(turn):
    global g_piece_list, g_total, g_not_matched

    boards = set()
    empty_board = chess.Board(None)
    empty_board.turn = turn
    tmp_piece_list = []
    boards.add(to_integer(empty_board, tmp_piece_list))
    
    total = 0
    not_matched = 0
   
    for piece in g_piece_list:
        boards, total, not_matched = fill_boards_with_piece(boards, piece, tmp_piece_list, total, not_matched)
        tmp_piece_list.append(piece)
    
    g_total += total
    g_not_matched += not_matched

Die Funktion `matchrate()` führt die Rechnung für beide Spieler durch und gibt die Matchrate als Konsolenausgabe zurück.

In [ ]:
def matchrate():
    matchrate_for_player(chess.WHITE)
    matchrate_for_player(chess.BLACK)
    percentage = (g_total - g_not_matched) / g_total * 100
    print("Matching Rate:" + str(percentage) + "%")

Die folgende Zelle füllt die Variable `g_all_boards` und führt die Rechnung durch. 

In [ ]:
g_all_boards = set.union(*g_s_n_sequence)
matchrate()

## Ergebnis der Validierung

Durch Nutzung dieses Notebooks kann festgestellt werden, dass die Endspieldaten für alle Situationen ohne Bauern optimal sind. Es werden 100% der gewinnbaren Situationen abgedeckt (`matching rate`). Die Genauigkeit (Züge zum Sieg) sind gleich zu den Endspieldaten von Syzygy und Gaviota.

Eine Besonderheit lässt sich beim Vergleich des Syzygy Tests mit dem Gaviota Test erkennen. Ab $S_2$ betrachtet Syzygy in der Situation "König gegen König und zwei Läufer" alle Situationen als falsch eingeordnet. Da Situationen aus $S_3$ jedoch in drei Halbzügen zum Sieg geführt werden können, ist diese Information offensichtlich falsch. Dieses Phänomen wurde nicht genauer untersucht und stattdessen der Vergleich mit Gaviota bevorzugt.

Der Sonderfall König, König Bauer, welcher als Einziger eine Änderung der Figuren auf dem Spielfeld enthält, kann von diesem Notebook nicht komplett validiert werden. Da die Spielzüge nach einer Wandlung des Bauern zur Dame optimal berechnet sind, wurde dieses Notebook nicht angepasst, um die Schritte des Bauern bis zur Umwandlung zu validieren.
